# **OJKGPT**

This is the Jupyter Notebook for the development of the 1st version of OJKGPT for the showcase demo capabilites to wealth team program. <br>
In this development, we use several data sources for enriching the RAG system of the chatbot. 

In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [4]:
from utils.models_definer import ModelName

# storing or not
OCR_TRESHOLD = 0.98
TOP_K = 3
NUM_BATCH_EVAL = 10
STORE = False
DELETE = False
EVAL = False
model_name = ModelName.AZURE_OPENAI
model_name_eval = ModelName.OPENAI

## **LLM Models Define**

In [5]:
from llama_index.core import Settings
from utils.models_definer import get_llm_and_embedding

# ollama/openai/azure_openai
llm,embedding_llm = get_llm_and_embedding(model_name=model_name)

llm_eval = get_llm_and_embedding(model_name=model_name_eval)[0]


Settings.llm = llm
Settings.embed_model = embedding_llm

## **Loading Documents**

### **Document Reader and Node Parser**

In [6]:
from utils.documents_reader import read_documents
from utils.node_parser import parse_nodes

path = './data'

if STORE:
    docs = read_documents(path=path, ocr_treshold=OCR_TRESHOLD)
    nodes = parse_nodes(documents=docs, llm=llm)

## **Indexing & Storing**

In [7]:
from utils.index_store import (store_vector_index, load_vector_index)

# store
if STORE:
    vector_index = store_vector_index(nodes=nodes,embed_model=embedding_llm, delete=DELETE)
# load
else:
    vector_index = load_vector_index()

Loading the vector index completed.


## **Querying**

### **Retriever**

In [8]:
vector_retriever = vector_index.as_retriever(similarity_top_k=TOP_K)

### **Query Engine**

In [9]:
from llama_index.postprocessor.colbert_rerank import ColbertRerank

colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

In [10]:
from llama_index.core.query_engine import RetrieverQueryEngine
import time

vector_query_engine = RetrieverQueryEngine.from_args(retriever=vector_retriever,llm=llm, node_postprocessors=[colbert_reranker])
time.sleep(2)

In [30]:
a = "Apa judul peraturan 7/33/PBI/2005?" # Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat
b = "Kapan surat edaran No. 15/26/DPbS mulai berlaku?" # 1 Agustus 2013.
c = "Siapa nama dan jabatannya yang menandatangani surat dengan nomor 1/SEOJK.04/2013?" # NURHAIDA, kepala eksekutif pengawas pasar modal
d = "Saya ingin menyelenggarakan kegiatan pasar modal berikan saya nomor surat yang membahas mengenai hal ini!" # Peraturan Pemerintah Nomor 12 Tahun 2004
e = "Berapa persen jaminan moneter pada tanggal 20 Agustus 1958?" # 7,3%
f = "Surat edaran nomor berapa yang mengatur bank umum syariah dan unit usaha syariah?" # 15/26/DPbS
g = "Apa kepanjangan dari PAPSI?" # Pedoman Akuntansi Perbankan Syariah Indonesia
h = "apa judul peraturan nomor 112/KMK.03/2001?" # Keputusan Menteri Keuangan tentang Pemotongan Pajak Penghasil Pasal 21 atas Penghasilan berupa Uang Pesangon, Uang Tebusan Pensiun, dan Tunjangan Hari Tua atau Jaminan Hari Tua
i = "Saya ingin membuat sistem informasi lembaga jasa keuangan, berikan nomor regulasi dari peraturan yang membahas tentang manejemen risiko nya!" # 4/POJK.05/2021
j = "Apa kepanjangan dari SWDKLLJ?" # Sumbangan Wajib Dana Kecelakaan Lalu Lintas Jalan
k = "Berapa nilai SWDKLLJ mobil sedan?" # Rp. 140.000
l = "Apa latar belakang dari peraturan NOMOR 4/POJK.05/2021?" # dalam bentuk list
m = "Apa itu LJKNB?" # Lembaga Jasa Keuangan Non Bank
n = "Apakah KMK Nomor 462/KMK.04/1998 masih berlaku" # tidak
o = "Apa itu Uang Pesangon?" # penghasilan yang dibayarkan oleh pemberi kerja kepada karyawan dengan nama dan dalam bentuk apapun sehubungan dengan berakhirnya masa kerja atau terjadi pemutusan  hubungan kerja, termasuk uang penghargaan masa kerja dan uang  ganti kerugian
p = "Apa itu CKPN?" # Cadangan Kerugian Penurunan Nilai.
q = "Kapan, dimana, dan oleh siapa surat nomor PER- 06/BL/2012 ditetapkan?" # Surat nomor PER-06/BL/2012 ditetapkan pada tanggal 22 November 2012 di Jakarta oleh Ketua Badan Pengawas Pasar Modal dan Lembaga Keuangan.
r = "Apa kepanjangan PSAK?" # Pernyataan Standar Akuntansi Keuangan
s = "Apa itu 'shahibul maal'?" # Pemilik dana pihak ketiga

query_str = a

In [31]:
from llama_index.core.response.notebook_utils import display_response

response_vector = vector_query_engine.query(query_str)
display_response(response_vector)

**`Final Response:`** Peraturan Bank Indonesia tentang Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat.

## **Evaluation**

In [32]:
import os

# check directory exist
if not os.path.exists("./json_data"):
    os.makedirs("./json_data")

In [33]:
from llama_index.core.evaluation import (
    generate_question_context_pairs,
    EmbeddingQAFinetuneDataset,
)
if EVAL:
    if STORE:
        qa_dataset = generate_question_context_pairs(
            nodes=nodes,
            llm=llm_eval,
            num_questions_per_chunk=2,
        )
        qa_dataset.save_json("./json_data/pg_eval_dataset.json")
    else:
        qa_dataset = EmbeddingQAFinetuneDataset.from_json("./json_data/pg_eval_dataset.json")

### **Retrieval Evaluation**

In [34]:
metrics = ["hit_rate", "mrr", "precision", "recall", "ap", "ndcg"]

In [35]:
from utils.evaluator import get_retrieval_eval_df

if EVAL:
    vector_retrieval_eval_results = await get_retrieval_eval_df(name=f"Top-{TOP_K} Eval", metrics=metrics, retriever=vector_retriever, qa_dataset=qa_dataset)
    display(vector_retrieval_eval_results)

### **Response Evaluation**

In [36]:
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
)

if EVAL:
    # model for evaluation
    relevancy_evaluator = RelevancyEvaluator(llm=llm_eval)
    faithfullness_evaluator = FaithfulnessEvaluator(llm=llm_eval)

#### **Faithfulness Evaluator**

Evaluate Response

In [37]:
from utils.evaluator import get_response_eval_df

if EVAL:
    vector_faithfullness_eval_result = faithfullness_evaluator.evaluate_response(response=response_vector, query=query_str)
    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=vector_faithfullness_eval_result))

Evaluate Source Nodes

In [38]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=faithfullness_evaluator)
    display(relevancy_eval_sources)

#### **Relevancy Evaluator**

Evaluate Response

In [39]:
if EVAL:
    relevancy_eval_result = relevancy_evaluator.evaluate_response(
        query=query_str, response=response_vector
    )

    display(get_response_eval_df(query=query_str, response=response_vector, eval_result=relevancy_eval_result))

Evaluate Source Nodes

In [40]:
from utils.evaluator import get_response_eval_sources_df

if EVAL:
    relevancy_eval_sources = get_response_eval_sources_df(query=query_str, response=response_vector, evaluator=relevancy_evaluator)
    display(relevancy_eval_sources)

#### **Batch Evaluator (Faithfulness, Relevancy, Correctness)**

In [41]:
from llama_index.core.evaluation import BatchEvalRunner
from utils.evaluator import get_batch_eval_results
from utils.evaluator import get_batch_eval_df

if EVAL:
    runner = BatchEvalRunner(
        {
            "faithfulness": faithfullness_evaluator, 
            "relevancy": relevancy_evaluator,
        },
        workers=8,
    )

    vector_eval_results = await get_batch_eval_results(runner=runner, qa_dataset=qa_dataset, query_engine=vector_query_engine, num_queries=NUM_BATCH_EVAL)
    display(get_batch_eval_df(vector_eval_results))

## **Build Agent**

In [42]:
# system_prompt = """Anda adalah chatbot yang dapat membantu menjawab pertanyaan tentang berbagai jenis regulasi di Indonesia.

# Anda HARUS selalu menjawab dengan BAHASA QUERY.
# Anda TIDAK DIBENARKAN berimajinasi.
# Anda HANYA DIBENARKAN menjawab pertanyaan berdasarkan dokumen yang telah Anda pelajari.
# JANGAN MENGGUNAKAN informasi dari luar dokumen yang telah Anda pelajari.
# Anda memiliki akses ke query tools untuk membantu Anda menemukan informasi yang relevan di basis data regulasi.
# Berdasarkan informasi konteks dan bukan pengetahuan sebelumnya, jawablah pertanyaan HARUS menggunakan query tools yang tersedia.
# Gunakan riwayat percakapan sebelumnya atau konteks di atas untuk berinteraksi dan membantu pengguna.

# Anda memiliki akses ke query tools berikut: 
# - **vector_tool**: Untuk menemukan konteks spesifik dari dokumen
# - **summary_tool**: Untuk menemukan ringkasan dokumen

# **Penjelasan Metadata:**
# Metadata dokumen mencakup informasi berikut:
# - **file_name**: Nama file dokumen
# - **title**: Judul dokumen
# - **sector**: Sektor yang dicakup oleh regulasi
# - **subsector**: Subsektor yang dicakup oleh regulasi
# - **regulation_type**: Jenis regulasi (misalnya, Surat Edaran OJK, Peraturan OJK)
# - **regulation_number**: Nomor regulasi
# - **effective_date**: Tanggal berlakunya regulasi

# ----------------------------------------------
# Query: {query_str}
# Jawaban:
# """

In [43]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
memory = ChatMemoryBuffer.from_defaults(
    token_limit=10000,
    chat_store=chat_store,
    chat_store_key="user1",
)

In [44]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.core.agent.legacy.react.base import ReActAgent
from llama_index.core.chat_engine import CondenseQuestionChatEngine

agent = CondenseQuestionChatEngine.from_defaults(
    llm=llm,
    memory=memory,
    query_engine=vector_query_engine,
    verbose=True,
)

In [45]:
response_agent = agent.chat(message=query_str)

Querying with: Apa judul peraturan 7/33/PBI/2005?


In [46]:
from llama_index.core.response.notebook_utils import display_response

display_response(response=response_agent)

**`Final Response:`** Peraturan Bank Indonesia tentang Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat.

In [47]:
from llama_index.core.response.notebook_utils import display_source_node

for node in response_agent.source_nodes:
    display_source_node(node, show_source_metadata=True)    

**Node ID:** 7785fd99-3d59-49ce-94ca-c7bd89fa14e4<br>**Similarity:** 0.7417713403701782<br>**Text:** PERATURAN BANK INDONESIA 
NOMOR : 7/33/PBI/2005 
TENTANG 
PENCABUTAN ATAS PERATURAN BANK INDONESI...<br>**Metadata:** {'file_name': 'ojk-ppbi-7_33_pbi_2005-22092005-pbi_tentang_pencabutan_atas_pbi_nomor_517pbi2003_tentang_persyaratan_dan_tata_cara_pelaksanaan_jaminan_pemerintahan__280_pdf.pdf', 'title': 'Peraturan Bank Indonesia tentang Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat', 'sector': 'Perbankan', 'subsector': 'Bank Umum,  BPR', 'regulation_type': 'PPBI', 'regulation_number': '7/33/PBI/2005', 'effective_date': '22 September 2005', 'retrieval_score': 0.9328267574310303}<br>

**Node ID:** 315f0d18-5912-4730-8a63-eca17c404ff4<br>**Similarity:** 0.7383741140365601<br>**Text:** - 3 - 
 
3. Undang-Undang Nomor 24 Tahun 2004 tentang Lembaga 
Penjamin Simpanan (Lembaran Negara...<br>**Metadata:** {'file_name': 'ojk-ppbi-7_33_pbi_2005-22092005-pbi_tentang_pencabutan_atas_pbi_nomor_517pbi2003_tentang_persyaratan_dan_tata_cara_pelaksanaan_jaminan_pemerintahan__280_pdf.pdf', 'title': 'Peraturan Bank Indonesia tentang Pencabutan atas Peraturan Bank Indonesia Nomor 5/17/PBI/2003 tentang Persyaratan dan Tata Cara Pelaksanaan Jaminan Pemerintah terhadap Kewajiban Pembayaran Bank Perkreditan Rakyat', 'sector': 'Perbankan', 'subsector': 'Bank Umum,  BPR', 'regulation_type': 'PPBI', 'regulation_number': '7/33/PBI/2005', 'effective_date': '22 September 2005', 'retrieval_score': 0.9303895235061646}<br>

**Node ID:** 37c0b43a-c17c-4249-9c82-238b32d3f0cd<br>**Similarity:** 0.6666749119758606<br>**Text:** II- 2
PP. No. : 12 Tahun 2004
Pasal I
Mengubah ketentuan Pasal 8 dalam Peraturan Pemerintah Nomor...<br>**Metadata:** {'file_name': 'ojk-peraturan_pemerintah-12_tahun_2004-02032004-pp_nomor_12_tahun_2004_tentang_perubahan_atas_peraturan_pemerintah_nomor_45_tahun_1995_pp_no_12_tahun_2004_pdf.pdf', 'title': 'Peraturan Pemerintah Nomor 12 Tahun 2004 tentang Perubahan atas Peraturan Pemerintah Nomor 45 Tahun 1995 tentang Penyelenggaraan Kegiatan di Bidang Pasar Modal', 'sector': 'Pasar Modal', 'subsector': 'Bursa Efek,  Perusahaan Efek,  Wakil Perusahaan Efek', 'regulation_type': 'Peraturan Pemerintah', 'regulation_number': '12 Tahun 2004', 'effective_date': '2 Maret 2004', 'retrieval_score': 0.9186574816703796}<br>

In [48]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)